# Content:
1. [Row-reduction/Elimination](#1.-Row-reduction/Elimination)
2. [Gaussian elimination followed by substitution](#2.-Gaussian-elimination-followed-by-substitution)
    > [2.1. Elimination: Transform of A to U](#2.1.-Elimination:-Transform-of-A-to-U)    
    > [2.2. Backward substitution (bottom to top)](#2.2.-Backward-substitution-(bottom-to-top))   
    > [2.3. Generalizing elimination followed by substitution](#2.3.-Generalizing-elimination-followed-by-substitution)
3. [LU decomposition and Cholesky decomposition](#3.-LU-decomposition-and-Cholesky-decomposition)    
4. [`scipy.linalg`](#4-`scipy.linalg`)
5. [Applications](#5.-Applications)

## 1. Row-reduction/Elimination

In the approach, you will augment the given matrix, ${\bf A}$, with the known vector, ${\bf b}$. Let's denote the augmented matrix as ${\bf A}_1=\left[ {\bf A} | {\bf b} \right]$. Now, you will perform a sequence of operations which will transform the augmented matrix into a new augmented matrix $\left[ {\bf I} | {\bf x} \right]$.
$$
\left[ {\bf A} | {\bf b} \right] \rightarrow \left[ {\bf I} | {\bf x} \right]
$$

If you perform the same set of operations on an indentity matrix, you will obtain the inverse of ${\bf A}$.

$$
\left[ {\bf A} | {\bf b} | {\bf I} \right] \rightarrow \left[ {\bf I} | {\bf x} | {\bf A}^{-1} \right]
$$

![boardwork005.jpg](../boardwork/boardwork005.jpg)

In [1]:
import numpy as np

A=np.array([[2,0,-1],[6,5,3],[2,-1,0]],float)
b=np.array([2,7,4],float)

#=== Augmented matrix 
A1=np.zeros([3,4])    # Declare a zero 3x4 matrix

print(A1,"\n===\n")

A1[0:3,0:3]=A.copy()  # Store the 'A' matrix in the first 3 columns
print(A1,"\n===\n")

A1[0:3,3]=b.copy()    # Store the 'b' vector in the 4th column

print('The augmented matrix [A|b] is:\n', A1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 
===

[[ 2.  0. -1.  0.]
 [ 6.  5.  3.  0.]
 [ 2. -1.  0.  0.]] 
===

The augmented matrix [A|b] is:
 [[ 2.  0. -1.  2.]
 [ 6.  5.  3.  7.]
 [ 2. -1.  0.  4.]]


#### Use the first row of A1 to eliminate the 'rest' (i.e. second and third elements) of the first column of A1

In [2]:
A1[1,:]=A1[1,:]-(A1[1,0]/A1[0,0])*A1[0,:]  # Transform second row
A1[2,:]=A1[2,:]-(A1[2,0]/A1[0,0])*A1[0,:]  # Transform third row

print('After subtracting row-1 from row-2 and row-3:\n', A1)

After subtracting row-1 from row-2 and row-3:
 [[ 2.  0. -1.  2.]
 [ 0.  5.  6.  1.]
 [ 0. -1.  1.  2.]]


#### Use the second row of A1 to eliminate the 'rest' of the second column of A1

In [3]:
A1[0,:]=A1[0,:]-(A1[0,1]/A1[1,1])*A1[1,:] # Transform first row
A1[2,:]=A1[2,:]-(A1[2,1]/A1[1,1])*A1[1,:] # Transform third row

print('After subtracting row-2 from row-1 and row-3:\n', A1)

After subtracting row-2 from row-1 and row-3:
 [[ 2.   0.  -1.   2. ]
 [ 0.   5.   6.   1. ]
 [ 0.   0.   2.2  2.2]]


#### Use the third row of A1 to eliminate the 'rest' of the third column of A1

In [4]:
A1[0,:]=A1[0,:]-(A1[0,2]/A1[2,2])*A1[2,:] # Transform first row
A1[1,:]=A1[1,:]-(A1[1,2]/A1[2,2])*A1[2,:] # Transform second row

print('After subtracting row-3 from row-1 and row-2:\n', A1)

After subtracting row-3 from row-1 and row-2:
 [[ 2.   0.   0.   3. ]
 [ 0.   5.   0.  -5. ]
 [ 0.   0.   2.2  2.2]]


#### Now scale all the rows

In [5]:
A1[0,:]=A1[0,:]/A1[0,0]
A1[1,:]=A1[1,:]/A1[1,1]
A1[2,:]=A1[2,:]/A1[2,2]

print('...and finally, we have the transformed augmented matrix:\n', A1)

...and finally, we have the transformed augmented matrix:
 [[ 1.   0.   0.   1.5]
 [ 0.   1.   0.  -1. ]
 [ 0.   0.   1.   1. ]]


Let's collect all the steps and make one piece of code!

In [6]:
import numpy as np

A=np.array([[2,0,-1],[6,5,3],[2,-1,0]],float)
b=np.array([2,7,4],float)

#=== Augmented matrix
A1=np.zeros([3,4],float)
A1[0:3,0:3]=A.copy()
A1[0:3,3]=b.copy()

print('The augmented matrix [A|b] is:\n', A1)

# Do all steps in row-reduction listed above using loops 
for i in range(0,3):
    for j in range(0,3):
        if i != j:
            A1[j,:]=A1[j,:]-(A1[j,i]/A1[i,i])*A1[i,:]

print('The augmented matrix [A|b] after row-reduction is:\n', A1)

# Scale the diagonal matrix
for i in range(0,3):
    A1[i,:]=A1[i,:]/A1[i,i]

x=np.zeros(3)
x=A1[:,3]
print('The solution is:\n',x)

The augmented matrix [A|b] is:
 [[ 2.  0. -1.  2.]
 [ 6.  5.  3.  7.]
 [ 2. -1.  0.  4.]]
The augmented matrix [A|b] after row-reduction is:
 [[ 2.   0.   0.   3. ]
 [ 0.   5.   0.  -5. ]
 [ 0.   0.   2.2  2.2]]
The solution is:
 [ 1.5 -1.   1. ]


## 2. Gaussian elimination followed by substitution 

Instead of the transformation $\left[ {\bf A} | {\bf b} \right] \rightarrow \left[ {\bf I} | {\bf x} \right]$, in Gaussian elimination, we perform the transformation only to the upper or the lower triangle of ${\bf A}$. 
$$
\left[ {\bf A} | {\bf b} \right] \rightarrow \left[ {\bf U} | {\bf c} \right]
$$
Following this, we determine the solution by backward or forward substitution.

![boardwork007.jpg](../boardwork/boardwork007.jpg)

In [7]:
import numpy as np

A=np.array([[2,1,1],[1,2,1],[1,1,2]],float)
b=np.array([1,2,3],float)

#=== Augmented matrix
A1=np.zeros([3,4],float)
A1[0:3,0:3]=A.copy()
A1[0:3,3]=b.copy()

print('The augmented matrix [A|b] is:\n', A1)

The augmented matrix [A|b] is:
 [[2. 1. 1. 1.]
 [1. 2. 1. 2.]
 [1. 1. 2. 3.]]


### 2.1. Elimination: Transform of A to U 

Use the first row of A1 to eliminate the 'rest' of the first column of A1

In [8]:
A1[1,:]=A1[1,:]-(A1[1,0]/A1[0,0])*A1[0,:]
A1[2,:]=A1[2,:]-(A1[2,0]/A1[0,0])*A1[0,:]
print('After subtracting row-1 from row-2 and row-3:\n', A1)

After subtracting row-1 from row-2 and row-3:
 [[2.  1.  1.  1. ]
 [0.  1.5 0.5 1.5]
 [0.  0.5 1.5 2.5]]


Use the second row of A1 to eliminate the 'rest' of the second column of A1

In [9]:
A1[2,:]=A1[2,:]-(A1[2,1]/A1[1,1])*A1[1,:]
print('After subtracting row-2 from row-3:\n', A1)

After subtracting row-2 from row-3:
 [[2.         1.         1.         1.        ]
 [0.         1.5        0.5        1.5       ]
 [0.         0.         1.33333333 2.        ]]


### 2.2. Backward substitution (bottom to top) 

In [10]:
U=np.zeros([3,3])
U[0:3,0:3]=A1[0:3,0:3].copy()
print('The U-form of matrix A is:\n', U)

The U-form of matrix A is:
 [[2.         1.         1.        ]
 [0.         1.5        0.5       ]
 [0.         0.         1.33333333]]


In [11]:
b=np.zeros([3])
b[0:3]=A1[0:3,3].copy()
print('The modified coefficient vector is:\n', b)

The modified coefficient vector is:
 [1.  1.5 2. ]


Now we have to solve the following equations.
$$
  2x+y+z=1 \\
    1.5y+0.5z = 1.5 \\
    1.33333333z=2
$$

Let's start by solving the last equation.
$$
1.33333333z=2 \Rightarrow z = 2/1.33333333
$$

In [12]:
x=np.zeros([3])
x[2]=b[2]/U[2,2]
print(x)

[0.  0.  1.5]


Now let's  solve the second equation
$$
1.5y+0.5z = 1.5
$$
by substituting the value of z=1.5
$$
1.5y+0.5(1.5)=1.5\\
\rightarrow 1.5y+0.75 = 1.5 \\
\rightarrow 1.5y=0.75\\
\rightarrow y = 0.75/1.5 = 0.5
$$

In [13]:
x[1]=(b[1]-U[1,2]*x[2])/U[1,1]
print(x)

[0.  0.5 1.5]


Now, the first equation
$$
  2x+y+z=1 \\
\rightarrow 2x+0.5+1.5=1\\
\rightarrow 2x=-1\\
\rightarrow x=-1/2
$$

In [14]:
x[0]=(b[0]-U[0,2]*x[1]-U[0,2]*x[2])/U[0,0]
print(x)

[-0.5  0.5  1.5]


### 2.3. Generalizing elimination followed by substitution 

![boardwork006.jpg](../boardwork/boardwork006.jpg)
![boardwork008.jpg](../boardwork/boardwork008.jpg)

In [15]:
import numpy as np

A=np.array([[2,1,1],[1,2,1],[1,1,2]],float) #NOTE: float
b=np.array([1,2,3],float)
x=np.zeros(3,float)

N=A.shape[0]

print(A)

print('\nTransformation: A -> U\n')

for k in range(0,N-1):
    for i in range(k+1,N):
        lam=A[k,i]/A[k,k]
        A[i,:]=A[i,:]-lam*A[k,:]
        b[i]=b[i]-lam*b[k]

print(A)

print(b)

print('\nBackward substitution to get x\n')

x[2]=b[2]/A[2,2]
for k in range(N-2,-1,-1):
    x[k]=b[k]
    for j in range(k+1,N):
        x[k]=x[k]-A[k,j]*x[j]
    x[k]=x[k]/A[k,k]

print(x)

[[2. 1. 1.]
 [1. 2. 1.]
 [1. 1. 2.]]

Transformation: A -> U

[[2.         1.         1.        ]
 [0.         1.5        0.5       ]
 [0.         0.         1.33333333]]
[1.  1.5 2. ]

Backward substitution to get x

[-0.5  0.5  1.5]


## 3. LU decomposition and Cholesky decomposition 

More often than not we will encounter a symmetric coefficient matrix, i.e., ${\bf A}^T={\bf A}$, where ${^T}$ denotes transpose. In such cases, Cholesky decomposition is the ideal option.

![boardwork010.jpg](../boardwork/boardwork010.jpg)

Any matrix can be written as ${\bf A}={\bf L}{\bf U}$. When ${\bf A}$ is symmetric, ${\bf U}$ is simply ${\bf L}^T$. Hence, we can write ${\bf A}={\bf L}{\bf L}^T$. Expanding the elements of ${\bf L}{\bf L}^T$ and equating to ${\bf A}$ shows that it is possible to write the elements of ${\bf L}$ in terms of the elements of ${\bf A}$ without having to perform sequential eliminations (or row-reductions).

![boardwork011.jpg](../boardwork/boardwork011.jpg)
![boardwork012.jpg](../boardwork/boardwork012.jpg)


---
Homework-5: Write a python function to solve a linear system of $N$ equations with $N$ unknowns. 
The function should perform Gaussian elimination if the matrix is not symmetric, and perform Cholesky decomposition when the matrix is symmetric.

---

## 4. `scipy.linalg`

Even though the previous problem asks you to write a program for your own version of Gaussian elimination and Cholesky decomposition, you can always use the Cholesky linear solver from the `scipy` library. 

To begin with, here is an example to show how to do perform a Cholesky decomposition.

In [16]:
import numpy as np
from scipy.linalg import cholesky
# See, https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.cholesky.html 

A=np.array([[2,1,1],[1,2,1],[1,1,2]],float) 

L = cholesky(A, lower=True)

print('\nGiven matrix A is\n')
print(A)

print('\nAfter A -> L, we have\n')
print(L)

LT=np.transpose(L)

print('\nL^T is\n')
print(LT)

print('\nL x L^T gives us the original matrix\n')
print(np.matmul(L,LT))


Given matrix A is

[[2. 1. 1.]
 [1. 2. 1.]
 [1. 1. 2.]]

After A -> L, we have

[[1.41421356 0.         0.        ]
 [0.70710678 1.22474487 0.        ]
 [0.70710678 0.40824829 1.15470054]]

L^T is

[[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]]

L x L^T gives us the original matrix

[[2. 1. 1.]
 [1. 2. 1.]
 [1. 1. 2.]]


## `scipy.linalg.cho_factor`  and `scipy.linalg.cho_solve` 

Now, let's see how to solve linear equations with Cholesky decomposition.

In [17]:
import numpy as np
from scipy.linalg import cho_factor, cho_solve
# See, https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.cho_solve.html

A=np.array([[2,1,1],[1,2,1],[1,1,2]],float) 
b=np.array([1,2,3],float)
x=np.zeros(3,float)

L, low = cho_factor(A)   # low is a logical output, 'true' if L is a lower triangular matrix
x = cho_solve((L, low), b)

print('\nThe solution is\n')
print(x)


The solution is

[-0.5  0.5  1.5]


We can check if the solution is correct by performing ${\bf A}{\bf x}$ and see it corresponds to ${\bf b}$. Or, even better is to see if ${\bf A}{\bf x}-{\bf b}$ corresponds to a zero vector.

In [18]:
print(np.matmul(A,x)-b)

[2.22044605e-16 0.00000000e+00 0.00000000e+00]


If we are working with very large matrix, it is hard to check all the elements and see if they are all zero. In that case, we can use the logical operator `numpy.allclose` which checks if two vectors are same elementwise and returns `True` or `False`.

In [19]:
np.allclose(np.matmul(A,x),b)
# See, https://numpy.org/doc/stable/reference/generated/numpy.allclose.html

True

## `scipy.linalg.solve`

If you want to work with general matrices that are not necessarily symmetric, you can use the general function `scipy.linalg.solve` which uses different algorithms based on the type of the matrix. 

In [20]:
import numpy as np
from scipy import linalg
# See, https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html#scipy.linalg.solve

A=np.array([[2,1,1],[3,2,1],[4,1,2]],float) 
b=np.array([1,2,3],float)
x=np.zeros(3,float)

print('\nGiven matrix is\n')
print(A)

print('\nKnown vector is\n')
print(b)

x = linalg.solve(A, b)
print('\nSolution is\n')
print(x)

print('\nIs the solution correct?\n')
check=np.allclose(np.matmul(A,x),b)
print(check)


Given matrix is

[[2. 1. 1.]
 [3. 2. 1.]
 [4. 1. 2.]]

Known vector is

[1. 2. 3.]

Solution is

[ 2. -1. -2.]

Is the solution correct?

True


## 5. Applications

xzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz![boardwork013.jpg](../boardwork/boardwork013.jpg)
![boardwork014.jpg](../boardwork/boardwork014.jpg)
![boardwork015.jpg](../boardwork/boardwork015.jpg)
![boardwork016.jpg](../boardwork/boardwork016.jpg)
![boardwork017.jpg](../boardwork/boardwork017.jpg)
![boardwork018.jpg](../boardwork/boardwork018.jpg)